# Purpose
Design an amplifier that can be used as the basis for a later Q-enhanced LNA design, similar to the one described in [Episode 1, Part 3 (of 3) in Radio Design 401, by MegawattKS](https://www.youtube.com/watch?v=43wQmYeqt44&t=1282s)

When looking for an integrated circuit (IC) solution, the Mini-circuits PSA4-5043+ or PGA-103+ are good candidates. They have a very low noise figure (NF) and are affordable and available.  A design with the PSA4-5043+ is shown [here](http://www.g4ddk.com/Anglian3lschematic.pdf) also used in the LNA4ALL.

Infineon has [dedicated FM LNA transistors](https://www.infineon.com/dgdl/Infineon-Design_Guide_for_low_noise_TR_in_FM_radio_FE-ApplicationNotes-v01_00-EN.pdf).  Alternatively, discrete wide band LNA designs [exist](https://www.qsl.net/va3iul/High_Frequency_LNA_schematics/High_Frequency_LNA_schematics.htm).


# Requirements
* 20dB gain at 98MHz
* 20MHz bandwidth, i.e. the whole FM band (88-108MHz)
* 50ohm input impedance (no matching network needed)
* 5V supply voltage
* Common-base stage followed by common-emitter (aka emitter-follower) stage

# Design
## Common-base stage
### Calculate I_c
Input impedance for a CB-stage is approximately 1/gm, where gm is the transconductance of the transistor. The transconductance is given by gm = Ic/Vt, where Ic is the collector current and Vt is the thermal voltage (approximately 26mV at room temperature).  From that, we can calculate the collector current needed to achieve the desired input impedance.
$$R_{in} = \frac{1}{g_{m}},g_{m}=\frac{I_{C}}{V_{T}}\Rightarrow I_{C} = \frac{V_{T}}{R_{in}}$$

In [1]:
# Calculat the required collector current for an input impedance of 50 ohm
V_T = 26e-3  # Thermal voltage at room temperature in volts
R_in = 50  # Input impedance in ohms
I_c = V_T / R_in
print(f"Required collector current (I_c): {I_c:.2e} A")

Required collector current (I_c): 5.20e-04 A


To handle larger input signals, Ic needs to be higher.  This results in a lower input impedance, which will require a matching network.

### Calculate R_C
From that, we can calculate the collector resistor value needed for the desired gain. The voltage gain of a common-base stage is given by:
$$A_{Vloaded} = \frac{A_{Vno-load}}{2}$$
$$A_{Vno-load} = g_{m}*R_{C}$$
$$R_{C}=\frac{2*A_{Vloaded}}{g_{m}}$$
Where A_Vunloaded is the desired voltage gain of the CB-stage. The CC-stage has a voltage gain of approximately 1, so the overall voltage gain of the amplifier is approximately equal to the voltage gain of the CB-stage.

In [2]:
G_T_dB = 20  # total gain in dB
G_T = 10 ** (G_T_dB / 20)  # Total voltage gain in linear scale
A_Vunloaded = 2* G_T # Unloaded voltage gain in volts
g_m = I_c / V_T  # Transconductance in S (Siemens)
R_C = A_Vunloaded / g_m
print(f"Collector resistance (R_C): {R_C:.2f} ohm")

Collector resistance (R_C): 1000.00 ohm


### Calculate biasing network

In [3]:
def calc_bias(Ic, Vbe, V_E, hfe):
    # Calculate biasing resistors
    Re = V_E / Ic # Emitter resistor
    print("\tEmitter resistor: {:.2f} ohm".format(Re))
    Ib = Ic / hfe # Base current
    V_R2 = Vbe + V_E # Voltage over R2
    I_R2 = 12 * Ib # Current through R2 for stable biasing
    R2 = V_R2 / I_R2 # Base resistor
    print("\tBottom bias resistor: {:.2f} ohm".format(R2))
    V_R1 = Vcc - V_R2 # Voltage over R1
    I_R1 = I_R2 + Ib # Current through R1
    R1 = V_R1 / I_R1 # Top bias resistor
    print("\tTop bias resistor: {:.2f} ohm".format(R1))
    return R1, R2, Re

def stability_check(R1, R2, Re, hfe):
    R_th = R1 * R2 / (R1 + R2) # Thevenin resistance

    # Bowick "Transistor Biasing" (ISBN: 978-0-7506-8518-4): As a practical rule of thumb for stable designs, the ratio R_th/Re should be less than 10.
    if(R_th / Re) < 10:
        print("\tStability check: OK")
    else:
        print("\tStability check: NOT OK")

    S = (hfe + 1) * (Re + R_th) / ((hfe+1) * Re + R_th) # Stability factor
    print("\tStability factor: {:.2f}".format(S))
    if S < 10:
        print("\tGood biasing")
    elif S < 20:
        print("\tAcceptable biasing")
    else:
        print("\tUnstable biasing")

Vcc = 5 # Supply voltage
Vbe = 0.8 # Base-emitter voltage
V_E = 0.7 # Assumed voltage over emitter resistor
hfe = 100 # Current gain

[R1, R2, Re] = calc_bias(I_c, Vbe, V_E, hfe)

stability_check(R1, R2, Re, hfe)

	Emitter resistor: 1346.15 ohm
	Bottom bias resistor: 24038.46 ohm
	Top bias resistor: 51775.15 ohm
	Stability check: NOT OK
	Stability factor: 11.77
	Acceptable biasing


We have now enough info to draw the first schematic. The DC-operating point analysis can be used to check our calculations.

<figure>
<img src="./doc/DC-operating_point.png" width="80%"/>
<figcaption>DC-operating point</figcaption>
</figure>

## Common-collector stage

For maximum output swing, the emitter should be biased at half the supply voltage. 

The bias resistors shouldn't be too small, because they will load the output of the CB-stage.  

## Image filter

In [4]:
import math 
def parallel_RLC(Rp, B, f_0):
    ω = 2 * math.pi * f_0
    Q = f_0 / B
    print("\tQuality factor Q = {:.2f}".format(Q))
    X_0 = Rp / Q
    L = X_0 / ω
    print("\tCalculated L = {:.2e} H".format(L))
    C = 1 / (ω * X_0)
    print("\tCalculated C = {:.2e} F".format(C))
    B = f_0 / Q
    print("\tCalculated B = {:.2e} Hz".format(B))
    return L, C, B

f = 98e6 # 98 MHz, center frequency of the FM broadcast band
B = 20e6 # 20 MHz, bandwidth (88-108 MHz)
L, C, B = parallel_RLC(R_C, B, f)

	Quality factor Q = 4.90
	Calculated L = 3.31e-07 H
	Calculated C = 7.96e-12 F
	Calculated B = 2.00e+07 Hz


Adding these values in the simulation resulted in an f_0 of 77.5MHz and a Q of 4.47.

In [5]:
f_real = 77.5e6 # 77.5 MHz
Q_real = 4.47
L = 330e-9 # 330 nH
ω = 2 * math.pi * f_real
X_0 = ω * L
Rp_real = X_0 * Q_real
print("\tCalculated Rp = {:.2e} ohm".format(Rp_real))
R_parasitic  = R_C * Rp_real / (R_C - Rp_real)
print("\tCalculated R_parasitic = {:.2e} ohm".format(R_parasitic))

# Now we can calculate the needed R_C for an equivalent R_p of 1000 ohm
R_p = 1000  # Target parallel resistance
R_C = R_parasitic * R_p / (R_parasitic - R_p)
print("\tCalculated R_C = {:.2e} ohm".format(R_C))

	Calculated Rp = 7.18e+02 ohm
	Calculated R_parasitic = 2.55e+03 ohm
	Calculated R_C = 1.65e+03 ohm


So, the new R_C in the simulation is 1600ohm.

In [6]:
L, C, B = parallel_RLC(R_C, B, f)

	Quality factor Q = 4.90
	Calculated L = 5.45e-07 H
	Calculated C = 4.84e-12 F
	Calculated B = 2.00e+07 Hz


The problem with this is that needed capacitor is smaller than the parasitic capacitance (even without taking into account the parasitic capacitance of the PCB).  

Let's keep the R_C of 1kohm and live with the reduced gain.  Using L = 222nH (WE 744918222) and C = 7.5pF, gives us center frequency of 98MHz, a peak gain of 18.8dB and a Q of 4.87.

We might also stock up on smaller value inductors.  These will needed bigger capacitors, but the Q will be higher.

<figure>
<img src="./doc/service_band_gain.png" width="80%"/>
<figcaption>Service band gain and input match</figcaption>
</figure>

# OIP3 : Third order intercept point
More info on [Analog Devices : IP3 and Intermodulation Guide](https://www.analog.com/en/resources/technical-articles/ip3-and-intermodulation-guide.html)

According to this application note, a +5dBm is ok for an LNA, but it's not good enough for the RF-frontend of a receiver.

Infineon's [Design guide for low-noise transistors in FM radio front ends](https://www.infineon.com/dgdl/Infineon-Design_Guide_for_low_noise_TR_in_FM_radio_FE-ApplicationNotes-v01_00-EN.pdf?fileId=8ac78c8c7e7124d1017f0227976a6c9d) use lower supply voltage (1.8V) which makes it harder to achieve a good OIP3.  They arrive at +3dBm.

The PSA4-5043+ has an OIP3 of +32dBm (5VDC supply).

In [7]:
# From LTSpice simulation:
VOUT_dbV = -32.4 # Output voltage in dBV of the fundamentals
ΔP_dB = 48.3 # difference in output power between the fundamental and the third harmonic
n = 3 # order of the harmonic

POUT_dBm = VOUT_dbV + 30 - 10 * math.log10(50) # Convert to dBm
OIP3 = POUT_dBm + ΔP_dB/(n-1)
print(f"Output IP3: {OIP3:.2f} dBm")

Output IP3: 4.76 dBm


# Measurements
## DC-operating point (L7 not placed)
| Signal | Simulation | Measurement |
|--------|------------|-------------|
| Vbe1 | 1.39V | 1.43V |
| Vc1 (L4 and L7 not placed) |4.24V | 4.10V |
| Ve1 | 0.62V | 0.72V |
| Vb2 | 3.12V | 3.16V |
| Ve2 | 2.30V | 2.39V |

## Input impedance

The output power of the NanoVNA will overdrive this amplifier.  To fix this, a 23dB (20+3) attenuator was added on port 0 and the output power was set to 2mA (=minimum output power).  Then the calibration was done again.

* R10 = 1.2kOhm, Zin = 42 Ohm
* R10 = 1.3kOhm, Zin = 47 Ohm
* R10 = 1.5kOhm, Zin = 51.6 Ohm

TODO : Instead of this figure, it would be better to show a Smith chart.
<figure>
<img src="./doc/meas_input_impedance.png" width="80%"/>
<figcaption>Input impedance</figcaption>
</figure>

## Gain with different inductor options
### Wuerth 744918222 (222nH)
Using R=1.5k, L=744918222, C=7.5pF → 71.6MHz, G=21.04dB, so the total load capacitance is 22.3pF.  We have only added 7.5pF, so the parasitic capacitance is 14.8pF.
Removing the capacitor gives us a gain of 21dB at 88.5MHz.  So we can never reach resonance at 98MHz with this inductor.  Why has the simulation not shown this? 

The copper pad area of the collector of the CB-amplifier is too large.  We might want to reduce the size of the 5V area as well.  After reducing the copper area, the gain peaks at 93.75MHz with a gain of 21.05dB.  Bandwidth is from 85.75MHz to 102.625MHz, so 16.875MHz. This gives us a Q of 5.56.  So the parallel RLC-circuit is loaded with 

In [8]:
# LC-calculator
import math
L = 169e-9  # Inductance in henries
f_0 = 98e6  # Resonant frequency in hertz

ω = 2 * math.pi * f_0  # Angular frequency in radians per second
C = 1 / (ω**2 * L)  # Capacitance in farads

print(f"Calculated capacitance for LC circuit: {C:.2e} F")

BW = 16.875e6  # Measured bandwidth in hertz
Q = f_0 / BW  # Quality factor
print(f"Calculated quality factor Q: {Q:.2f}")

Xl = ω * L  # Inductive reactance in ohms
Rp = Q * Xl # Parallel resistance in ohms
print(f"Calculated parallel resistance Rp: {Rp:.2e} ohm")

# So we actually need a BW of 20MHz at a center frequency of 98MHz
ω = 2 * math.pi * 98e6
L = 1 / (ω**2 * C)  # Calculate inductance for the desired center frequency
print(f"Calculated inductance for desired center frequency: {L:.2e} H")

Calculated capacitance for LC circuit: 1.56e-11 F
Calculated quality factor Q: 5.81
Calculated parallel resistance Rp: 6.04e+02 ohm
Calculated inductance for desired center frequency: 1.69e-07 H


<figure>
<img src="./doc/no_LNA_FM_spectrum.bmp" width="80%"/>
<figcaption>FM spectrum without LNA</figcaption>
</figure>
<figure><img src="./doc/LNA_FM_spectrum.bmp" width="80%"/>
<figcaption>FM spectrum with LNA</figcaption>
</figure>
When connected to the Yaesu FT-65, the LNA is able to pick up the FM stations in the area.  However it doesn't pick up more stations than the FT-65 does without the LNA.  

In [9]:
# dBµV to µV conversion
def dBµV_to_µV(dBµV):
    """Convert dBµV to µV."""
    µV = 10 ** ((dBµV / 20) + 6)
    print(f"Converted {dBµV} dBµV to {µV:.2f} µV")
    return µV

# dBm to μV conversion
def dBm_to_μV(dBm):
    """Convert dBm to μV."""
    μV = 10 ** ((dBm + 107) / 20)
    print(f"Converted {dBm} dBm to {μV:.2f} μV")
    return μV

dBm_to_μV(-55)  # Example conversion

Converted -55 dBm to 398.11 μV


398.1071705534973


### WE 744918220, 206nH
This gives us a center frequency of 94.375MHz, a gain of 21.05dB and a bandwidth of (86.375Mhz to 103.125MHz).  While we can add a capacitor to achieve resonance at 98MHz, the bandwidth will be too low.

### Abracon AIAC4125CR169JT, 169nH
That would require a total capacitance of 15.6pF for max. gain at 98MHz.  Without parallel cap, max. gain is at 103.5MHz.  So we already have 14pF on the board.
Let's add 1.5pF.  Maximum gain of 20.7dB is at 98.75MHz (close enough) and the -3dB bandwidth is from 91.2MHz to 106.85MHz.

<figure>
<img src="./doc/nanovna-h4_2025-07-29_15-27-22.png" width="800"/>
<figcaption>tested with 20dB attenuator between nanoVNA and input port</figcaption>
</figure>
<figure>
<img src="./doc/nanovna-h4_2025-07-29_15-47-32.png" width="800"/>
<figcaption>tested with 20dB attenuator between nanoVNA and input port</figcaption>
</figure>

<figure>
<img src="./doc/nanovna-h4_2025-07-29_16-46-36.png" width="800"/>
<figcaption>tested with 23dB attenuator between nanoVNA and input port (very noisy because of small signal levels)</figcaption>
</figure>

Alternative inductors to try out: 
* WE 744918217 (169nH)
* Walsin 

<figure>
<img src="../LNA/doc/IMG_20250729_211953_138.jpg" width="800"/>
<figcaption>PCB with LNA</figcaption>
</figure>
The collector area of the CB-stage had too much coupling with to GND.  Scratching away the copper area below the inductor as much as possible, reduced that.

# Preselector filter followed by the LNA
More info on [Preselector filter for FM band](../preselector_filter/Filters.ipynb)

<figure>
<img src="./doc/nanovna-h4_2025-07-30_20-36-39.png" width="800"/>
<figcaption>Preselector filter followed by the LNA</figcaption>
</figure>
<figure>
<img src="./doc/nanovna-h4_2025-07-30_20-41-13.png" width="800"/>
<figcaption>Preselector filter followed by the LNA</figcaption>
</figure>
<figure>
<img src="../LNA/doc/IMG_20250730_204305_936.jpg" width="800"/>
<figcaption>Image of test setup for the above measurements</figcaption>
</figure>

# Q-enhanced LNA
## Design
1. The relationship between the feedback resistor and the gain bandwidth is exponential.
2. The feedback amplifies the Q of the circuit.  The higher the open-loop Q, the lower the amount of feedback needed to achieve the same closed loop Q.  Higher Q can be achieved using inductors with a smaller inductance.
3. Tuning across the band, changes the tuning capacitor and hence also the Q of the circuit.  This means that the feedback resistor needs to be adjusted to keep the Q constant across the band.

MegawattKS suggests starting with an open loop Q of 50.  With an R_C of 1k, we already simulated that R_p is 718ohm.

In [10]:
f_0 = 98e6
Q = 50
R_p = 718
X_0 = R_p / Q
ω = 2 * math.pi * f_0
L = X_0 / ω
print("\tCalculated L = {:.2e} H".format(L))
C = 1 / (ω * X_0)
print("\tCalculated C = {:.2e} F".format(C))

	Calculated L = 2.33e-08 H
	Calculated C = 1.13e-10 F


In [11]:
def calc_C(f0, L):
    # Calculate the capacitance needed for a parallel resonant circuit
    C = 1 / (4 * math.pi**2 * f0**2 * L)
    return C
def calc_Q(f0, C, R):
    return R * 2 * math.pi * f0 * C

C1 = calc_C(88e6, 100e-9) # 98 MHz, 330 nH
Q1 = calc_Q(88e6, C1, R_C) # 98 MHz, 330 nH
Cm = calc_C(98e6, 100e-9) # 98 MHz, 330 nH
Qm = calc_Q(98e6, Cm, R_C) # 98 MHz, 330 nH
C2 = calc_C(108e6, 100e-9) # 98 MHz, 330 nH
Q2 = calc_Q(108e6, C2, R_C) # 98 MHz, 330 nH
dQ = (Q2 - Q1)/Qm # 98 MHz, 330 nH


# Noise figure

The simulated noise figure is 3.5dB.  That's not really fantastic, but it's not bad either.